# 1. Activate the following environments before running this notebook

>1a. lmi_ai.env

>1b. python virtual environments

ensure that you are currently in this directory: 'LMI_AI_Solutions/object_detectors/examples/thinFilm/yolov5'

# 2. Prepare datasets for training

## 2a. Data annotation - VGG annotator

## 2b. Convert json annotations to csv format

In [ ]:
%run -m via_json_to_csv -- --data_path ./data/allImages_1024 --output_fname labels.csv

## 2c. Resize images and annotations

In [ ]:
%run -m resize_images_with_csv -- -i ./data/allImages_1024 --out_imsz 640,640 -o ./data/2022-01-08_640

## 2d. Convert to YOLO file format

In [ ]:
%run ../../../yolov5/preprocess/convert_data_to_yolo.py -i ./data/2022-01-08_640 -o ./data/2022-01-08_640_yolo

# 3. Download Pre-trained COCO model

In [ ]:
%rm -r ./pretrained-models
%mkdir ./pretrained-models
!wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt -P ./pretrained-models

# 4. Ensure the order of class names are same

In [ ]:
!cat ./data/2022-01-08_640_yolo/class_map.json

In [ ]:
!cat ./config/2022-01-08_640.yaml

# 5. Train the model

In [ ]:
%run ../../../yolov5/train.py --img 640 --batch 16 --epoch 600 --data ./config/2022-01-08_640.yaml --weights ./pretrained-models/yolov5s.pt --project ./training --name 2022-01-08_640_temp --exist-ok

# 6. Monitor the training process using Tensorboard

In [ ]:
!tensorboard --logdir ./training/2022-01-08_640

# 7. Run inference after training

In [ ]:
%run ../../../yolov5/detect.py --source ./data/2022-01-08_640_yolo/images --weights ./training/2022-01-08_640/weights/best.pt --conf-thres 0.5 --img 640 --project ./validation --name 2022-01-08_640 --save-csv --exist-ok

# 8. Evaluate the trained model

In [ ]:
#calculate the IOU (intersection over union)
%run -m iou_from_csv -- --manual_csv ./data/2022-01-08_640/labels.csv --model_csv ./validation/2022-01-08_640/preds.csv --data_dir ./data/2022-01-08_640 --output_dir ./validation/2022-01-08_640_iou